> library dependencies

+ itunes API searcher
    + ```$ pip install pyitunes```

In [9]:
import itunes
import pandas as pd 
import matplotlib.pyplot as plt
import bs4 as bs
import urllib.request

### this is a wrapper for the [iTunes Search Api]()

In [10]:
# for example
item = itunes.search (query='star wars')
print (item)

[<Feature-Movie>: Star Wars: The Force Awakens, <Feature-Movie>: Star Wars: The Phantom Menace, <Feature-Movie>: Star Wars: The Empire Strikes Back, <Tv-Episode>: R2 Come Home, <Feature-Movie>: Star Wars: Return of the Jedi, <Feature-Movie>: Star Wars: A New Hope, <Feature-Movie>: Star Wars: Attack of the Clones, <Feature-Movie>: Star Wars: Revenge of the Sith, <Podcast>: Rebel Force Radio: Star Wars Podcast, <Song>: Star Wars, <Tv-Episode>: Bombad Jedi, <Tv-Episode>: An Old Friend, <Tv-Episode>: Weapons Factory, <Tv-Episode>: The Lawless, <Tv-Episode>: Citadel Rescue, <Tv-Episode>: Point of No Return, <Tv-Episode>: Bounty, <Tv-Episode>: The Box, <Tv-Episode>: Ambush, <Tv-Episode>: Rising Malevolence, <Tv-Episode>: The Unknown, <Tv-Episode>: Rookies, <Tv-Episode>: Sacrifice, <Tv-Episode>: Defenders of Peace, <Tv-Episode>: Conspiracy, <Tv-Episode>: Downfall of a Droid, <Tv-Episode>: Shadow of Malevolence, <Tv-Episode>: Hostage Crisis, <Tv-Episode>: Mystery of the Thousand Moons]


## This searches for podcasts, instead of music albums,
### and prroduces

### Set the query term and Run All Cells
### ⇩


In [58]:
# queryterm = 'cystic fibrosis'
queryterm = 'infectious disease'
# queryterm = 'genetic'
# queryterm = 'covid19'
# queryterm = 'foreclosure'
podsearch = itunes.search(query=queryterm, media='podcast')

In [59]:
for p in podsearch:
    print (p)

<Podcast>: Infectious Diseases Society of America
<Podcast>: Persiflagers Infectious Disease Puscast
<Podcast>: Emerging Infectious Diseases
<Podcast>: Infectious Disease
<Podcast>: Persiflagers Infectious Disease Puscast
<Podcast>: Breakpoints
<Podcast>: The Open Forum Infectious Diseases Podcast
<Podcast>: PeerView Infectious Diseases CME/CNE/CPE Audio Podcast
<Podcast>: Infectious Disease & Antibiotic Resistance
<Podcast>: Infectious Disease Dynamics
<Podcast>: Disease State - Infectious
<Podcast>: PeerVoice Immunology & Infectious Disease Video
<Podcast>: Infectious Questions : An Infectious Diseases Public Health Podcast
<Podcast>: Infectious Diseases
<Podcast>: Infectious Diseases and Pandemics (audio)
<Podcast>: Podcasts – PKIDs Blog
<Podcast>: PeerVoice Immunology & Infectious Disease Audio
<Podcast>: Infectious Diseases and Pandemics (video)
<Podcast>: PeerView Infectious Diseases CME/CNE/CPE Video Podcast
<Podcast>: The History of Infectious Diseases


In [60]:
# let's looks at what gets returned for each item in that list of Podcast search results
print (vars(podsearch[0]))

{'id': 519582740, 'name': 'Infectious Diseases Society of America', 'url': 'https://podcasts.apple.com/us/podcast/infectious-diseases-society-of-america/id519582740?uo=4', '_release_date': None, 'artwork': {'30': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/30x30bb.jpg', '60': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/60x60bb.jpg', '600': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/600x600bb.jpg'}, 'json': {'wrapperType': 'track', 'kind': 'podcast', 'collectionId': 519582740, 'trackId': 519582740, 'artistName': 'IDSA', 'collectionName': 'Infectious Diseases Society of America', 'trackName': 'Infectious Diseases Society of America', 'collectionCensoredName': 'Infectious Diseases Society of America', 'trackCensoredName': 'Infectious Diseas

### ⇪ A little wierd, looks like most of what we want is packed into
### a "JSON" attribute, so...there it is.

- - -

# So let's lookup some stuff about the podcast in question

+ no. of individual posts / episodes per year since publishing
+ length posts (minutes)
+ rating per channel +/or track, if available

### first for # posts / episodes / media URL

In [61]:
print (f'{len(podsearch)} channels\n')
for channel in podsearch:
    print (f'{channel.json["trackCount"]} podcasts in "{channel.json["collectionName"]}" (with id = {channel.id})')
    print (f'The last post in this channel was {channel.json["releaseDate"]}, and iTunes page lising all podcast episodes:')
    print (f'{channel.json["collectionViewUrl"]}\n')

20 channels

54 podcasts in "Infectious Diseases Society of America" (with id = 519582740)
The last post in this channel was 2020-05-23T07:30:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/infectious-diseases-society-of-america/id519582740?uo=4

300 podcasts in "Persiflagers Infectious Disease Puscast" (with id = 79930757)
The last post in this channel was 2020-05-17T08:25:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/persiflagers-infectious-disease-puscast/id79930757?uo=4

50 podcasts in "Emerging Infectious Diseases" (with id = 212828612)
The last post in this channel was 2020-05-21T15:56:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/emerging-infectious-diseases/id212828612?uo=4

49 podcasts in "Infectious Disease" (with id = 415333788)
The last post in this channel was 2016-03-15T10:31:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.co

## Fine.
### Uinfortunately, that 'iTunes Collection Preview' page only lists the first few episodes for the channel
### But podcast publisher's feed (not Apple) includes ALL podcasts 

In [89]:
channel_number = []
channel_name = []
channel_episodecount = []
channel_itunespreview = []
channel_feed = []

In [93]:
# "feedUrl" is not consistenly present, so handle missing Attributes

# definition to make write podcast channels to an array
def writeKeys(channel):
    channel_number.append(channel.id)
    channel_name.append(channel.json["collectionName"])
    channel_episodecount.append(channel.json["trackCount"])
    channel_itunespreview.append(channel.json["collectionViewUrl"])
    # feedURL is not consistent, check if present
    try:
       channel_feed.append(channel.json["feedUrl"])
    except KeyError as e:
        print (f'{e} does not exist in this channel')
        channel_feed.append(None)

In [94]:
for channel in podsearch:
    writeKeys(channel)

'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel


In [95]:
len(channel_name)

40

In [102]:
# create an "Channels" dataframe
data = {'ID': channel_number,
        'CHANNEL_NAME': channel_name,
        'EPISODE_COUNT': channel_episodecount,
        'ITUNES_PREVIEW_URL': channel_itunespreview,
        'PUBLISHER_FEED': channel_feed
       }
channels = pd.DataFrame(data,columns=['ID','CHANNEL_NAME','EPISODE_COUNT','ITUNES_PREVIEW_URL','PUBLISHER_FEED'])
channels

,ID,CHANNEL_NAME,EPISODE_COUNT,ITUNES_PREVIEW_URL,PUBLISHER_FEED
0,519582740,Infectious Diseases Society of America,54,https://podcasts.apple.com/us/podcast/infectio...,https://idsocietypodcast.libsyn.com/rss
1,79930757,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
2,212828612,Emerging Infectious Diseases,50,https://podcasts.apple.com/us/podcast/emerging...,https://www2c.cdc.gov/podcasts/createrss.asp?t...
3,415333788,Infectious Disease,49,https://podcasts.apple.com/us/podcast/infectio...,http://wwwf.imperial.ac.uk/imedia/itunes_colle...
4,1153935086,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
5,1470308447,Breakpoints,23,https://podcasts.apple.com/us/podcast/breakpoi...,https://pinecast.com/feed/sidp
6,1368469324,The Open Forum Infectious Diseases Podcast,28,https://podcasts.apple.com/us/podcast/the-open...,https://feeds.blubrry.com/feeds/the_ofid.xml
7,275853748,PeerView Infectious Diseases CME/CNE/CPE Audio...,16,https://podcasts.apple.com/us/podcast/peerview...,https://c.peerview.com/podcast/xml/infectiousd...
8,621579678,Infectious Disease & Antibiotic Resistance,7,https://podcasts.apple.com/us/podcast/infectio...,None
9,833385368,Infectious Disease Dynamics,53,https://podcasts.apple.com/us/podcast/infectio...,http://rss.sms.cam.ac.uk/rss/collection/153939...


In [103]:
channels.PUBLISHER_FEED[0]

'https://idsocietypodcast.libsyn.com/rss'

## Next: create an "Episodes" dataframe
### Constructed by keys in this 'dict'-like feed of each individual podcast episode

In [106]:
# what are the fields?
request = urllib.request.Request(channels.PUBLISHER_FEED[0],headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'})
source = urllib.request.urlopen(request)

# this is an XML feel
# inspection reveals that each podcast is encapsulated in <item> tags
soup = bs.BeautifulSoup(source,'lxml')
rows = soup.find_all('item')
print (f'This podcast channel has {len(rows)} episodes, and here is one episode from that feed:\n')
print (rows[0])

This podcast channel has 54 episodes, and here is one episode from that feed:

<item>
<title>Coronavirus: Educating ID Learners (May 23, 2020)</title>
<itunes:title>Coronavirus: Educating ID Learners (May 23, 2020)</itunes:title>
<pubdate>Sat, 23 May 2020 07:30:00 +0000</pubdate>
<guid ispermalink="false"></guid>
<link/>
<itunes:image href="https://ssl-static.libsyn.com/p/assets/c/5/7/9/c579e9b8cbaa3bb6/REVISED_1400x1400_3.png"></itunes:image>
<description>ID medical educators and students share how they’re adapting to the COVID-19 pandemic. This episode includes guest moderator Dr. Anna Person of Vanderbilt School of Medicine, and panelists Dr. Brian Schwartz of the University of California San Francisco, medical student Cassandra Thanh, &amp; Drs. Vera Luther and Kelsie Peirre of Wake Forest University.]]&gt;</description>
<content:encoded>ID medical educators and students share how they’re adapting to the COVID-19 pandemic. This episode includes guest moderator Dr. Anna Person of Va

In [107]:
def checkEpisodes(url):
    #since it's a request from the messy web, let's add some error handling
    try:
        request = urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'})
        source = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print (f'Failed {e}: {url}')
        return None
    try:
        soup = bs.BeautifulSoup(source,'lxml')
        episodes = soup.find_all('item')
    except AttributeError as e:
        return None
    return episodes

In [111]:
for x in channels.PUBLISHER_FEED:
    print (f'Working on {x}:')
    if x!=None:
        print("Has it")
    else:
        print("Nope, doesn't have it")
#     p = checkEpisodes(x)
#     print (f'{len(p)}\n')

Working on https://idsocietypodcast.libsyn.com/rss:
Has it
Working on http://www.pusware.com/podcast.xml:
Has it
Working on https://www2c.cdc.gov/podcasts/createrss.asp?t=a&c=49:
Has it
Working on http://wwwf.imperial.ac.uk/imedia/itunes_collections/view/infectious-disease.rss:
Has it
Working on http://www.pusware.com/podcast.xml:
Has it
Working on https://pinecast.com/feed/sidp:
Has it
Working on https://feeds.blubrry.com/feeds/the_ofid.xml:
Has it
Working on https://c.peerview.com/podcast/xml/infectiousdiseases_audio.xml:
Has it
Working on None:
Has it
Working on http://rss.sms.cam.ac.uk/rss/collection/1539399?feedType=itunes:
Has it
Working on http://api.delvenetworks.com/rest/organizations/ea495bf9c05048b88058f9803f6b94be/channels/9f38606cfa8f4f549c8900913a662c4d/media.itunesrss:
Has it
Working on https://c.peervoice.com/podcast/xml/rheumatology_video.xml:
Has it
Working on https://infectious.libsyn.com/rss:
Has it
Working on https://www.icgeb.org/itunesu/Infectious-Diseases.xml:
H

In [110]:
"""
        if (len(rows) > 0):
            print (f'This podcast channel has {len(rows)} episodes')
            for r in rows:
                title = r.find('itunes:title')
                print (title.text)
                pubdate = r.find('pubdate')
                print (pubdate.text)
                duration = r.find('itunes:duration')
                print (duration.text+'\n')
                
title = []
pubdate = []
description = []
duration = []
for r in rows:
    title = r.find('title')
    print (title.text)
    pubdate = r.find('pubdate')
    print (pubdate.text+ '\n')
   description = r.find('itunes:summary')
   print (description.text+ '\n')
    duration = r.find('itunes:duration')
    print (duration.text)
"""

"\ntitle = []\npubdate = []\ndescription = []\nduration = []\nfor r in rows:\n    title = r.find('title')\n    print (title.text)\n    pubdate = r.find('pubdate')\n    print (pubdate.text+ '\n')\n   description = r.find('itunes:summary')\n   print (description.text+ '\n')\n    duration = r.find('itunes:duration')\n    print (duration.text)\n"